## Builds a model for vectorizing the raw data (apply it once on train and once on test) :
* pivot from the initial feature_name:feature_value form to a vector
* handle dummy variables: translate categoric variables into N-1 dummy variables (The model is based on categories in train data)
* handle time-series variables: reduce them in several hard-coded methods
* fill missing values with train data means, and normalize to z-scores with train data std


In [1]:
from IPython.display import display

import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from vectorizing_funcs import *

In [2]:
df = pd.read_csv('../train_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
df.head()

,SubjectID,form_name,feature_name,feature_value,feature_unit,feature_delta
0,533,Demographic,Gender,F,NaN,0.0
1,533,Demographic,Age,65,NaN,0.0
2,533,Demographic,Race,White,NaN,0.0
3,533,ALSHX,onset_delta,-1023,NaN,0.0
4,533,ALSHX,diag_delta,-44,NaN,0.0


# Build metadata: assign features to vectorizing functions
funcs_to_features arrays define pairs of funcs (can be a list of functions or a single one) and features that should get these functions calculated. Overlapping is allowed.

There is a list for time-series functions (as described before) and for dummy functions. Both are inverted to feature_to_funcs maps.

In [3]:
    
all_feature_metadata = invert_func_to_features(ts_funcs_to_features, "ts")
all_feature_metadata.update(invert_func_to_features(dummy_funcs_to_features, "dummy"))

## Learn to_dummies model
Which kind of categories do we have available in our train data?

In [4]:
all_feature_metadata = learn_to_dummies_model(df, all_feature_metadata)

##Vectorize `train` data 

In [5]:

vectorized, all_feature_metadata = vectorize(df, all_feature_metadata, debug=True)
vectorized.head()


family_ALS_hist
weight
Gender
Age
respiratory_rate
pulse
Race
mouth
hands
onset_delta
Albumin
ALSFRS_Total
temperature
respiratory
bp_systolic
BMI
bp_diastolic
Creatinine
fvc_percent


,family_ALS_hist_last,weight_pct_diff,weight_mean_slope,weight_mean,weight_median,weight_std,F,M,Age_last,respiratory_rate_pct_diff,...,Creatinine_last,Creatinine_mean_slope,Creatinine_mean,Creatinine_median,Creatinine_std,fvc_percent_pct_diff,fvc_percent_mean_slope,fvc_percent_mean,fvc_percent_median,fvc_percent_std
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,NaN,-0.000108,0.004337,51.000000,51.00,0.547723,1,0,65,0.000000,...,79.56,0.000000,79.56,79.56,0.000000,-0.007220,-0.554290,73.443432,73.443432,4.703307
649,NaN,0.000156,0.035425,74.566667,74.40,0.960902,1,0,48,0.001171,...,53.04,0.271247,50.83,53.04,4.420000,-0.000863,-0.108080,94.369387,93.262709,2.738655
1234,NaN,0.000227,0.012698,95.050000,94.85,0.714143,0,1,38,0.002857,...,88.40,0.000000,88.40,88.40,0.000000,0.002728,0.872154,71.093086,69.782838,10.343727
2492,NaN,0.000000,0.000000,90.100000,90.10,0.000000,0,1,63,0.000000,...,70.72,0.260000,66.30,66.30,6.250824,-0.001989,-0.235351,114.325670,114.325670,5.658230
2956,NaN,-0.000059,-0.032251,65.100000,65.15,0.469042,1,0,63,0.000000,...,61.88,0.038268,55.25,53.04,4.420000,0.001571,0.312641,89.763630,89.988226,5.657501


In [6]:
vectorized.columns

Index([u'family_ALS_hist_last', u'weight_pct_diff', u'weight_mean_slope',
       u'weight_mean', u'weight_median', u'weight_std', u'F', u'M',
       u'Age_last', u'respiratory_rate_pct_diff',
       u'respiratory_rate_mean_slope', u'respiratory_rate_mean',
       u'respiratory_rate_median', u'respiratory_rate_std', u'pulse_pct_diff',
       u'pulse_mean_slope', u'pulse_mean', u'pulse_median', u'pulse_std',
       u'Asian', u'Black', u'Hispanic', u'Other', u'Unknown', u'White',
       u'mouth_pct_diff', u'mouth_last', u'mouth_mean_slope', u'mouth_mean',
       u'mouth_median', u'mouth_std', u'hands_pct_diff', u'hands_last',
       u'hands_mean_slope', u'hands_mean', u'hands_median', u'hands_std',
       u'onset_delta_last', u'Albumin_pct_diff', u'Albumin_last',
       u'Albumin_mean_slope', u'Albumin_mean', u'Albumin_median',
       u'Albumin_std', u'ALSFRS_Total_pct_diff', u'ALSFRS_Total_last',
       u'ALSFRS_Total_mean_slope', u'ALSFRS_Total_mean',
       u'ALSFRS_Total_median', u'AL

## Filling empty values with means and normalizing
- NOTE that we have to use the `train` data means and std

In [7]:
train_data_means = vectorized.mean()
train_data_std = vectorized.std()            
normalized, all_feature_metadata = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
normalized.head()


,family_ALS_hist_last,weight_pct_diff,weight_mean_slope,weight_mean,weight_median,weight_std,F,M,Age_last,respiratory_rate_pct_diff,...,Creatinine_last,Creatinine_mean_slope,Creatinine_mean,Creatinine_median,Creatinine_std,fvc_percent_pct_diff,fvc_percent_mean_slope,fvc_percent_mean,fvc_percent_median,fvc_percent_std
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,-0.232289,-0.002066,-1.620202,-1.617145,-0.435513,1.322796,-1.322796,0.941297,-0.169340,...,0.563661,0.175924,0.547815,0.558684,-1.276692,-1.864018,-1.318057,-0.499089,-0.507078,-0.064608
649,0,0.218932,0.466682,-0.086445,-0.100773,-0.163334,1.322796,-1.322796,-0.558473,0.169753,...,-0.863703,0.449074,-1.117773,-0.958224,-0.357095,-0.037980,-0.158877,0.750278,0.671602,-0.568227
1234,0,0.340632,0.124007,1.246644,1.224432,-0.325885,-0.755549,0.755549,-1.440691,0.658048,...,1.039449,0.175924,1.060304,1.064321,-1.276692,0.993322,2.387613,-0.639414,-0.724779,1.381257
2492,0,-0.047759,-0.067460,0.924490,0.916621,-0.796321,-0.755549,0.755549,0.764854,-0.169340,...,0.087873,0.437748,-0.220918,-0.199770,0.023815,-0.361454,-0.489506,1.941751,1.924245,0.180177
2956,0,-0.149486,-0.553742,-0.702551,-0.700194,-0.487343,1.322796,-1.322796,0.764854,-0.169340,...,-0.387915,0.214460,-0.861529,-0.958224,-0.357095,0.661195,0.934090,0.475295,0.476864,0.179990


## Pickle all metadata we will need to use later when applying vectorizer

In [8]:
pickle.dump( all_feature_metadata, open('../all_feature_metadata.pickle', 'wb') )
pickle.dump( train_data_means, open('../train_data_means.pickle', 'wb') )
pickle.dump( train_data_std, open('../train_data_std.pickle', 'wb') )


## Apply model on `train`,  `test` 


In [9]:

for t in ["train", "test"]:
    df = pd.read_csv('../' + t + '_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
    vectorized, _ = vectorize(df, all_feature_metadata)
    normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
    print t, normalized.shape
    normalized.to_csv('../' + t + '_data_vectorized.csv' ,sep='|')

normalized.head()

train (1777, 83)
test (600, 83)


,family_ALS_hist_last,weight_pct_diff,weight_mean_slope,weight_mean,weight_median,weight_std,F,M,Age_last,respiratory_rate_pct_diff,...,Creatinine_last,Creatinine_mean_slope,Creatinine_mean,Creatinine_median,Creatinine_std,fvc_percent_pct_diff,fvc_percent_mean_slope,fvc_percent_mean,fvc_percent_median,fvc_percent_std
SubjectID,,,,,,,,,,,,,,,,,,,,,
750059,0,0.300212,0.033383,2.120464,2.078067,-0.146931,-0.755549,0.755549,-0.029142,0.000000,...,-0.381457,0.142725,-0.078882,0.069064,-0.340451,0.000000,0.000000,0.000000,0.000000,0.000000
750094,0,1.333537,0.740484,-0.672179,-0.677513,0.258353,1.322796,-1.322796,0.853076,1.387573,...,-0.381457,-0.248462,-0.513686,-0.445724,0.010049,0.061534,-0.279429,0.018545,0.013536,-0.796276
750148,0,0.148987,-1.612401,-1.344999,-1.390337,0.949332,1.322796,-1.322796,1.117741,-0.876510,...,0.533520,0.692215,0.879956,0.927044,0.201766,0.089290,0.108626,0.851140,0.872872,-0.746680
750195,0,-0.554141,-0.505208,1.422364,1.412358,-0.377100,-0.755549,0.755549,1.117741,-0.169340,...,2.466813,-4.275089,2.426941,2.581229,-0.214833,0.209846,0.121899,0.169511,0.158915,-1.270254
750406,0,2.046137,0.686441,0.486274,0.378763,0.991207,1.322796,-1.322796,0.500188,-1.031203,...,1.515237,0.332099,1.700915,1.569957,-0.357095,-0.146617,0.007964,-0.195316,-0.148445,-0.172067
